In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import TimestampType
import pyspark.sql.functions as F
import folium
from folium import plugins
from IPython.display import clear_output
import pandas as pd

spark = SparkSession.builder.getOrCreate()

In [2]:
# 2
olist_geolocation_dataset = spark.read.options(header='True', inferSchema='True', delimiter=',') \
                            .csv("data/olist_geolocation_dataset.csv")

In [3]:
my_map = folium.Map()
world = folium.Map(
    zoom_start=11,
    location=[-23.54, -46.63], prefer_canvas=True)

In [4]:
world.save('Map_sell.html')

In [5]:
lok_sell = olist_geolocation_dataset.select("geolocation_lat","geolocation_lng")

In [6]:
lok_sell.show()

+-------------------+-------------------+
|    geolocation_lat|    geolocation_lng|
+-------------------+-------------------+
| -23.54562128115268| -46.63929204800168|
|-23.546081127035535| -46.64482029837157|
| -23.54612896641469| -46.64295148361138|
|  -23.5443921648681| -46.63949930627844|
|-23.541577961711493| -46.64160722329613|
|-23.547762303364266| -46.63536053788448|
|-23.546273112412678| -46.64122516971552|
|-23.546923208436723|  -46.6342636964915|
|-23.543769055769133| -46.63427784085132|
|-23.547639550320632| -46.63603162315495|
|-23.547325128224376| -46.63418378613892|
| -23.53841810407414|-46.634778375266734|
|-23.546435343326205| -46.63383023397196|
|-23.548945985189434| -46.63467113292871|
|-23.545187340816042| -46.63785524104107|
|-23.546081127035535| -46.64482029837157|
|-23.541883009983316| -46.63991946670314|
|-23.541389521053937|-46.629899087812184|
| -23.54693540437998| -46.63658792659698|
|-23.545884279214015|-46.643163191240035|
+-------------------+-------------

In [7]:
df2 = lok_sell.dropDuplicates()
df2.count()

718463

In [8]:
df2.count()/lok_sell.count()

0.7183459096167325

#### We have no value Null

In [9]:
print(f"Null values in geolocation_lat: {lok_sell.where(F.col('geolocation_lat').isNull()).count()}")
print(f"Null values in geolocation_lng: {lok_sell.where(F.col('geolocation_lng').isNull()).count()}")

Null values in geolocation_lat: 0
Null values in geolocation_lng: 0


In [18]:
# lok_samp = lok_sell
lok_samp = lok_sell.sample(0.01)
lok_samp.count()

10060

#### How we can speed up this loop?

In [ ]:
i = 0
x = lok_samp.count()
for row in range(lok_samp.count()):
    i += 1
    folium.CircleMarker(
        location=[lok_samp.collect()[row]['geolocation_lat'], lok_samp.collect()[row]['geolocation_lng']],
        radius=3,
#         popup='My Birthplace',
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=1
    ).add_to(world)
    clear_output(wait=True)
    print(f"{i}/{x}")

#### Why pandas works more than 100 times faster than pyspark?

In [19]:
pd_lok_samp = lok_samp.toPandas()

In [20]:
i = 0
x = len(pd_lok_samp.index)
for row in range(len(pd_lok_samp.index)):
    i += 1
    folium.CircleMarker(
        location=[pd_lok_samp.at[row,'geolocation_lat'], pd_lok_samp.at[row,'geolocation_lng']],
        radius=1,
#         popup='My Birthplace',
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=1
    ).add_to(world)
    clear_output(wait=True)
    print(f"{i}/{x}")

10060/10060


In [21]:
world.save('Map_sell.html')